In [ ]:
from pathlib import Path
import shutil
from tqdm import tqdm
from ultralytics import YOLO
import cv2
from pathlib import Path

In [ ]:
model_name = "yolo11n.pt"
image_directory = Path("images")
out_directory = Path("results")

exts = {".jpg", ".jpeg", ".png", ".bmp"}
image_paths = [p for p in image_directory.iterdir() if p.suffix.lower() in exts]

In [ ]:
model = YOLO(model_name)
# img = cv2.imread(image_directory) # 1 afbeelding

In [ ]:
SHOW       = True           # zet True als je elke image wil tonen
for img_path in image_paths:
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"Skip (niet leesbaar): {img_path}")
        continue

    results = model(img)  # detectie
    r = results[0]

    # Console output
    names = r.names
    for box in r.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(float, box.xyxy[0].tolist())
        print(f"{img_path.name}: {names[cls]} {conf:.2f} [{x1:.0f},{y1:.0f},{x2:.0f},{y2:.0f}]")

    # Annoteren + opslaan
    annotated = r.plot()
    out_path = out_directory / f"{img_path.stem}_det{img_path.suffix}"
    cv2.imwrite(str(out_path), annotated)

    if SHOW:
        cv2.imshow("YOLOv11", annotated)
        cv2.waitKey(0)

if SHOW:
    cv2.destroyAllWindows()

print("Klaar Resultaten staan in:", out_directory)

In [ ]:
# Paden moeten wijzigen want dit was lokaal
SOURCE_DIR = Path(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\CARLA.v20-carla_v20.yolov11")
TARGET_DIR = Path(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\Carla_acc8")

SRC_CLASSES = [
    'bike', 'motobike', 'person',
    'traffic_light_green', 'traffic_light_orange', 'traffic_light_red',
    'traffic_sign_30', 'traffic_sign_60', 'traffic_sign_90',
    'vehicle'
]

MAP_TO_TARGET = {
    'bike': 5, 'motobike': 4, 'person': 3, 'bus':2, 'truck':1,
    'traffic_light_green': 6, 'traffic_light_orange': 6, 'traffic_light_red': 6,
    'traffic_sign_30': 7, 'traffic_sign_60': 7, 'traffic_sign_90': 7,
    'car': 0

In [ ]:
from pathlib import Path
import shutil
from tqdm import tqdm
from ultralytics import YOLO
import cv2
from pathlib import Path

In [ ]:
model_name = "yolo11n.pt"
image_directory = Path("images")
out_directory = Path("results")

exts = {".jpg", ".jpeg", ".png", ".bmp"}
image_paths = [p for p in image_directory.iterdir() if p.suffix.lower() in exts]

In [ ]:
model = YOLO(model_name)
# img = cv2.imread(image_directory) # 1 afbeelding

In [ ]:
SHOW       = True           # zet True als je elke image wil tonen
for img_path in image_paths:
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"Skip (niet leesbaar): {img_path}")
        continue

    results = model(img)  # detectie
    r = results[0]

    # Console output
    names = r.names
    for box in r.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        x1, y1, x2, y2 = map(float, box.xyxy[0].tolist())
        print(f"{img_path.name}: {names[cls]} {conf:.2f} [{x1:.0f},{y1:.0f},{x2:.0f},{y2:.0f}]")

    # Annoteren + opslaan
    annotated = r.plot()
    out_path = out_directory / f"{img_path.stem}_det{img_path.suffix}"
    cv2.imwrite(str(out_path), annotated)

    if SHOW:
        cv2.imshow("YOLOv11", annotated)
        cv2.waitKey(0)

if SHOW:
    cv2.destroyAllWindows()

print("Klaar Resultaten staan in:", out_directory)

In [ ]:
# Paden moeten wijzigen want dit was lokaal
SOURCE_DIR = Path(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\CARLA.v20-carla_v20.yolov11")
TARGET_DIR = Path(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\Carla_acc8")

SRC_CLASSES = [
    'bike', 'motobike', 'person',
    'traffic_light_green', 'traffic_light_orange', 'traffic_light_red',
    'traffic_sign_30', 'traffic_sign_60', 'traffic_sign_90',
    'vehicle'
]

MAP_TO_TARGET = {
    'bike': 5, 'motobike': 4, 'person': 3,
    'traffic_light_green': 6, 'traffic_light_orange': 6, 'traffic_light_red': 6,
    'traffic_sign_30': 7, 'traffic_sign_60': 7, 'traffic_sign_90': 7,
    'vehicle': 0
}

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp"}

In [ ]:
def find_image(src_images_dir: Path, stem: str):
    for ext in IMG_EXTS:
        p = src_images_dir / f"{stem}{ext}"
        if p.exists():
            return p
    return None

In [ ]:
def remap_split(split: str, stats):

    # bronmappen
    src_img_dir = SOURCE_DIR / split / "images"
    src_lbl_dir = SOURCE_DIR / split / "labels"

    # doelmappen (jij hebt deze al aangemaakt)
    dst_img_dir = TARGET_DIR / "images" / ("train" if split == "train" else "val")
    dst_lbl_dir = TARGET_DIR / "labels" / ("train" if split == "train" else "val")

    lbl_files = list(src_lbl_dir.glob("*.txt"))

    for lbl_path in tqdm(lbl_files, desc=f"Processing {split}"):
        stem = lbl_path.stem
        img_path = find_image(src_img_dir, stem)

        if img_path is None:
            stats["missing_image"] += 1
            continue

        lines_out = []
        raw = lbl_path.read_text().strip().splitlines()

        for line in raw:
            parts = line.split()
            if len(parts) != 5:
                continue
            src_cls_idx = int(parts[0])
            src_name = SRC_CLASSES[src_cls_idx]

            if src_name not in MAP_TO_TARGET:
                continue

            tgt_cls = MAP_TO_TARGET[src_name]
            parts[0] = str(tgt_cls)
            lines_out.append(" ".join(parts))

            stats["per_class"][tgt_cls] = stats["per_class"].get(tgt_cls, 0) + 1

        # Image kopiëren
        shutil.copy2(img_path, dst_img_dir / img_path.name)

        # Labelbestand schrijven (mag leeg zijn)
        (dst_lbl_dir / lbl_path.name).write_text("\n".join(lines_out) + ("\n" if lines_out else ""))

        stats["images"] += 1
        if lines_out:
            stats["labeled_images"] += 1
        else:
            stats["empty_labels"] += 1


In [ ]:
stats = {"images":0, "labeled_images":0, "empty_labels":0, "missing_image":0, "per_class":{}}

# 1600 training images
# 264 validation images

splits = []
if (SOURCE_DIR / "train").exists(): splits.append("train")
if (SOURCE_DIR / "test").exists():   splits.append("test")
if (SOURCE_DIR / "valid").exists(): splits.append("valid")  # Roboflow  'valid'

for split in splits:
    remap_split(split, stats)

print("\n=== Samenvatting ===")
print(stats)

In [ ]:
from ultralytics import YOLO

model = YOLO(model_name)
model.train(
    data=r"C:/Users/Khatc/Desktop/DistributedAI/Project/Code_Train/dataset/Carla_acc8/data_acc8.yaml",
    epochs=2,        # korte rooktest
    imgsz=640,
    batch=4,
    device="cpu"
)


IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp"}

In [ ]:
def find_image(src_images_dir: Path, stem: str):
    for ext in IMG_EXTS:
        p = src_images_dir / f"{stem}{ext}"
        if p.exists():
            return p
    return None

In [ ]:
def remap_split(split: str, stats):

    # bronmappen
    src_img_dir = SOURCE_DIR / split / "images"
    src_lbl_dir = SOURCE_DIR / split / "labels"

    # doelmappen (jij hebt deze al aangemaakt)
    dst_img_dir = TARGET_DIR / "images" / ("train" if split == "train" else "val")
    dst_lbl_dir = TARGET_DIR / "labels" / ("train" if split == "train" else "val")

    lbl_files = list(src_lbl_dir.glob("*.txt"))

    for lbl_path in tqdm(lbl_files, desc=f"Processing {split}"):
        stem = lbl_path.stem
        img_path = find_image(src_img_dir, stem)

        if img_path is None:
            stats["missing_image"] += 1
            continue

        lines_out = []
        raw = lbl_path.read_text().strip().splitlines()

        for line in raw:
            parts = line.split()
            if len(parts) != 5:
                continue
            src_cls_idx = int(parts[0])
            src_name = SRC_CLASSES[src_cls_idx]

            if src_name not in MAP_TO_TARGET:
                continue

            tgt_cls = MAP_TO_TARGET[src_name]
            parts[0] = str(tgt_cls)
            lines_out.append(" ".join(parts))

            stats["per_class"][tgt_cls] = stats["per_class"].get(tgt_cls, 0) + 1

        # Image kopiëren
        shutil.copy2(img_path, dst_img_dir / img_path.name)

        # Labelbestand schrijven (mag leeg zijn)
        (dst_lbl_dir / lbl_path.name).write_text("\n".join(lines_out) + ("\n" if lines_out else ""))

        stats["images"] += 1
        if lines_out:
            stats["labeled_images"] += 1
        else:
            stats["empty_labels"] += 1


In [ ]:
stats = {"images":0, "labeled_images":0, "empty_labels":0, "missing_image":0, "per_class":{}}

splits = []
if (SOURCE_DIR / "train").exists(): splits.append("train")
if (SOURCE_DIR / "val").exists():   splits.append("val")
if (SOURCE_DIR / "valid").exists(): splits.append("valid")  # Roboflow  'valid'

for split in splits:
    remap_split(split, stats)

print("\n=== Samenvatting ===")
print(stats)

In [ ]:
from ultralytics import YOLO

model = YOLO(model_name)
model.train(
    data=r"C:/Users/Khatc/Desktop/DistributedAI/Project/Code_Train/dataset/Carla_acc8/data_acc8.yaml",
    epochs=2,        # korte rooktest
    imgsz=640,
    batch=4,
    device="cpu"
)



# Current version

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")      # pre-trained COCO backbone
select_classes = [0, 1, 5, 6, 9, 10, 15, 16, 17, 20, 22, 24, 25]
model.train(
    data="C:\\Users\\Khatc\\Desktop\\DistributedAI\\Project\\Code_Train\\dataset\\Carla.v1i.yolov11_using\\data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    classes=select_classes,
)
print(model.trainer.save_dir)

In [ ]:
# Later:
results = model.predict(
    source="C:\\Users\\Khatc\\Desktop\\DistributedAI\\Project\\Code_Train\\dataset\\Carla.v1i.yolov11_using\\test\\images",
    classes=[0, 1, 5, 6, 9, 10, 15, 16, 17, 20,22, 24, 25]  # bv. alleen bepaalde klassen tonen
)

In [ ]:
print(results)

In [ ]:
"""
for i, r in enumerate(results):
    cls_ids = r.boxes.cls.int().tolist()          # class indices in deze image
    unique_ids = sorted(set(cls_ids))
    class_names = [model.names[c] for c in unique_ids]

    print(f"Image {i}:")
    print("  class ids  :", unique_ids)
    print("  class names:", class_names)
"""
import os
for i, r in enumerate(results):
    # Volledig pad van de afbeelding
    img_path = r.path  # of r.orig_path als .path niet bestaat
    filename = os.path.basename(img_path)

    cls_ids = r.boxes.cls.int().tolist()          # class indices in deze image
    unique_ids = sorted(set(cls_ids))
    class_names = [model.names[c] for c in unique_ids]

    print(f"Image {i}: {filename}")
    print(f"  full path  : {img_path}")
    print("  class ids  :", unique_ids)
    print("  class names:", class_names)

In [ ]:
from ultralytics import YOLO

model = YOLO(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\runs\detect\train3\weights\best.pt")

metrics = model.val(
    data=r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\Carla.v1i.yolov11_using\data.yaml",
    split="test",   # of "val" als je de validatieset wilt
    imgsz=640,
    batch=16,
    classes=[0, 1, 5, 6, 9, 10, 15, 16, 17, 20, 22, 24, 25]  # alleen deze klassen evalueren
)


# Procentuele accuraatheid van de klassen die wij geselecteerd hebben

In [ ]:
print("mAP50-95 overall:", metrics.box.map)      # gemiddelde mAP (0.5:0.95)
print("mAP50 overall   :", metrics.box.map50)    # mAP op IoU=0.5
print("Per-class mAP50 :", metrics.box.maps)     # lijst met mAP50 per klasse


# Maakt een plot en saved dit van de % accuraatheid

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO

model = YOLO(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\runs\detect\train3\weights\best.pt")

metrics = model.val(
    data=r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\Carla.v1i.yolov11_using\data.yaml",
    split="test",
    imgsz=640,
    batch=16
)

per_class_map50 = metrics.box.maps
overall_map50 = metrics.box.map50
class_names = [model.names[i] for i in range(len(per_class_map50))]

plt.figure(figsize=(14, 6))
x = np.arange(len(per_class_map50))
plt.bar(x, per_class_map50)
plt.axhline(overall_map50, linestyle="--", linewidth=1.5, label=f"overall mAP50 = {overall_map50:.3f}")
plt.xticks(x, class_names, rotation=90)
plt.ylabel("mAP@0.5")
plt.title("Per-class mAP@0.5")
plt.legend()
plt.tight_layout()


plt.savefig("map_per_class.png")   # file will be in the same folder as your script
print("Saved plot to map_per_class.png")

# optional:
plt.show()


# Maakt een confusion matrix

In [ ]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ---- JOUW BESTAANDE CODE ----
model = YOLO(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\runs\detect\train3\weights\best.pt")

metrics = model.val(
    data=r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\Carla.v1i.yolov11_using\data.yaml",
    split="test",
    imgsz=640,
    batch=16,
    classes=[0, 1, 5, 6, 9, 10, 15, 16, 17, 20, 22, 24, 25]
)

# ---- CONFUSION MATRIX FILTEREN OP GEKOZEN KLASSEN ----

# Volledige confusion matrix (27 x 27)
cm = metrics.confusion_matrix.matrix

# Zorg dat het een numpy array is
if not isinstance(cm, np.ndarray):
    cm = cm.cpu().numpy()

# Klassen die je wilt houden
keep_classes = [0, 1, 5, 6, 9, 10, 15, 16, 17, 20, 22, 24, 25]

# Submatrix pakken (alleen deze rijen en kolommen)
cm_sub = cm[np.ix_(keep_classes, keep_classes)]

# Namen van de classes (optioneel, voor mooie as-labels)
class_names = [model.names[i] for i in keep_classes]

# ---- PLOT VAN DE GEFILTERDE CONFUSION MATRIX ----
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm_sub,
    annot=True,
    fmt=".0f",
    xticklabels=class_names,
    yticklabels=class_names
)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion matrix (alleen geselecteerde klassen)")
plt.tight_layout()
plt.show()



In [ ]:
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 1. Klassen die je wilt behouden
keep_classes = [0, 1, 5, 6, 9, 10, 15, 16, 17, 20, 22, 24, 25]

# 2. Model laden en val draaien
model = YOLO(r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\runs\detect\train3\weights\best.pt")

metrics = model.val(
    data=r"C:\Users\Khatc\Desktop\DistributedAI\Project\Code_Train\dataset\Carla.v1i.yolov11_using\data.yaml",
    split="test",
    imgsz=640,
    batch=16,
    # let op: 'classes' hier bepaalt wél welke objecten mee tellen,
    # maar Ultralytics houdt de confusion matrix nog steeds op volledige nc-grootte.
    classes=keep_classes
)

# 3. Volledige confusion matrix ophalen
cm_full = metrics.confusion_matrix.matrix  # dit is meestal een torch.Tensor

# Naar numpy omzetten (als het een tensor is)
cm_full = np.array(cm_full)

print("Volledige matrix shape:", cm_full.shape)  # bv. (27, 27)

# 4. Submatrix maken voor alleen de gewenste klassen
idx = np.array(keep_classes, dtype=int)
cm_sub = cm_full[np.ix_(idx, idx)]

print("Subset matrix shape:", cm_sub.shape)      # zou (13, 13) moeten zijn

# 5. Optioneel: DataFrame met labels (handig voor debug/inspectie)
class_names_full = model.names  # dict: {id: "naam"}
class_names_sub = [class_names_full[i] for i in keep_classes]

df_cm = pd.DataFrame(cm_sub, index=class_names_sub, columns=class_names_sub)
print(df_cm)  # tekstuele confusion matrix

# 6. Plotten van de subset confusion matrix
plt.figure(figsize=(10, 8))
plt.imshow(cm_sub, interpolation='nearest')
plt.title("Confusion matrix (alleen geselecteerde klassen)")
plt.colorbar()

tick_marks = np.arange(len(class_names_sub))
plt.xticks(tick_marks, class_names_sub, rotation=45, ha="right")
plt.yticks(tick_marks, class_names_sub)

# getallen in de cellen schrijven
thresh = cm_sub.max() / 2.0 if cm_sub.max() > 0 else 0.5
for i in range(cm_sub.shape[0]):
    for j in range(cm_sub.shape[1]):
        plt.text(
            j, i, int(cm_sub[i, j]),
            horizontalalignment="center",
            color="white" if cm_sub[i, j] > thresh else "black"
        )

plt.ylabel("True class")
plt.xlabel("Predicted class")
plt.tight_layout()
plt.savefig("confusion_matrix_subset.png", dpi=300)
plt.show()
